In [39]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import pprint as pp

In [40]:
base_url = 'https://hh.ru'
search = '/search/vacancy'

In [41]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'}

In [42]:
def zp(text):
    text = text.replace('\u202f', '')
    text_list = text.split()
    ot = '0'
    do = '0'
    if text.find('от') != -1:
        ot = text_list[text_list.index('от')+1]
    if text.find('до') != -1:
        do = text_list[text_list.index('до')+1]
    if text.find('–') != -1: 
        ot = text_list[text_list.index('–')-1]
        do = text_list[text_list.index('–')+1]
    val = text_list[-1]
    
    return ot, do, val

In [43]:
def get_maxpage():
    params = {'search_field':'name', 'text':inp_vacancy, 'clusters':'true', 'ored_clusters':'true', 'enable_snippets':'true', 'customDomain': '1', 'hhtmFrom': 'vacancy_search_list'}
    response = requests.get(base_url + search, headers = headers, params = params)
    dom = BeautifulSoup(response.text, 'html.parser')
    pages = dom.find_all('a', {'class':"bloko-button"})
    max_page = int(pages[-2].find('span').text)
    
    return max_page

Введите вакансию:

In [44]:
inp_vacancy = 'Системный администратор'

In [45]:
max_page = get_maxpage()

In [46]:
res_list = []
for num_page in np.arange(max_page):
    params = {'search_field':'name', 'text':inp_vacancy, 'clusters':'true', 'ored_clusters':'true', 'enable_snippets':'true', 'customDomain': '1', 'page': num_page, 'hhtmFrom': 'vacancy_search_list'}
    response = requests.get(base_url + search, headers = headers, params = params)
    dom = BeautifulSoup(response.text, 'html.parser')
    conteiners = dom.find_all('div', {'class':"vacancy-serp-item"})
    for vac in conteiners:
        link_tag = vac.find('a', {'data-qa':"vacancy-serp__vacancy-title"}, recursive = 'False') 
        money = vac.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
        if money is None:
            res_list = np.append(res_list, [link_tag.text, link_tag['href'], None, '0', '0', None])
        else:
            ot, do, val = zp(money.text)
            res_list = np.append(res_list, [link_tag.text, link_tag['href'], money.text, ot, do, val])    
            
result_df = pd.DataFrame(res_list.reshape((int(len(res_list)/6),6)), columns = ['Вакансия', 'Ссылка', 'Зарплата', 'От', 'До', 'Валюта'])  
result_df[['От', 'До']] = result_df[['От', 'До']].astype(int)

In [47]:
result_df.head(10)

,Вакансия,Ссылка,Зарплата,От,До,Валюта
0,Системный администратор,https://hh.ru/vacancy/51773794?from=vacancy_se...,150 000 – 160 000 руб.,150000,160000,руб.
1,Системный администратор,https://hh.ru/vacancy/52832179?from=vacancy_se...,от 60 000 руб.,60000,0,руб.
2,Системный администратор в поиск Яндекс,https://hh.ru/vacancy/52349868?from=vacancy_se...,до 50 000 руб.,0,50000,руб.
3,Системный администратор L1,https://hh.ru/vacancy/51626975?from=vacancy_se...,от 1 500 бел. руб.,1500,0,руб.
4,Системный администратор в поиск Яндекс,https://hh.ru/vacancy/52349869?from=vacancy_se...,до 50 000 руб.,0,50000,руб.
5,IT-специалист/Системный администратор,https://hh.ru/vacancy/52593175?from=vacancy_se...,90 000 – 90 000 руб.,90000,90000,руб.
6,Начинающий системный администратор macOS / Linux,https://hh.ru/vacancy/52605239?from=vacancy_se...,от 85 000 руб.,85000,0,руб.
7,Системный администратор сетей связи,https://hh.ru/vacancy/50539871?from=vacancy_se...,None,0,0,None
8,Системный администратор,https://hh.ru/vacancy/52595430?from=vacancy_se...,None,0,0,None
9,Системный администратор,https://hh.ru/vacancy/51883839?from=vacancy_se...,70 000 – 70 000 руб.,70000,70000,руб.


In [48]:
response.url

'https://hh.ru/search/vacancy?search_field=name&text=%D0%A1%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%BD%D1%8B%D0%B9+%D0%B0%D0%B4%D0%BC%D0%B8%D0%BD%D0%B8%D1%81%D1%82%D1%80%D0%B0%D1%82%D0%BE%D1%80&clusters=true&ored_clusters=true&enable_snippets=true&customDomain=1&page=39&hhtmFrom=vacancy_search_list'

In [49]:
params = {'search_field':'name', 'text':inp_vacancy, 'clusters':'true', 'ored_clusters':'true', 'enable_snippets':'true', 'customDomain': '1', 'hhtmFrom': 'vacancy_search_list'}
response = requests.get(base_url + search, headers = headers, params = params)
dom = BeautifulSoup(response.text, 'html.parser')
pages = dom.find_all('a', {'class':"bloko-button"})

In [50]:
int(pages[-2].find('span').text)

40